### Single model run
Obtain an optimised posterior parameter set from the selected analysis. Then illustrate results with basic plots, including the matrix scaling.

### Running over Colab
Uncomment the commands in the following cell to install in Colab.
Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment and then proceed to the following cells.

### Generating documentation
Running the final cell will take several minutes and will generate the corresponding PDF documentation.
This can be downloaded from the Files section of the sidebar.

In [ ]:
# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.9 numpy==1.24.3 kaleido

In [ ]:
import pandas as pd
import pymc as pm
import arviz as az

from estival.sampling.tools import idata_to_sampleiterator

from aust_covid.constants import PRIMARY_ANALYSIS, RUN_IDS, set_project_base_path
from aust_covid.inputs import get_ifrs
from aust_covid.model import build_model
from aust_covid.calibration import get_priors, get_targets
from aust_covid.plotting import plot_single_run_outputs, plot_example_model_matrices, plot_infection_processes
from emutools.tex import DummyTexDoc
from emutools.utils import load_param_info

project_paths = set_project_base_path("../")
SUPPLEMENT_PATH = project_paths["SUPPLEMENT_PATH"]
RUNS_PATH = project_paths["RUNS_PATH"]

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch main
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method("forkserver")
except:
    on_colab = False

In [ ]:
dummy_doc = DummyTexDoc()
param_info = load_param_info()
param_info["value"].update(get_ifrs(dummy_doc))
parameters = param_info["value"].to_dict()
epi_model = build_model(dummy_doc, param_info["abbreviations"], mobility_ext=True, cross_ref=False)
priors = get_priors(False, param_info["abbreviations"], DummyTexDoc())
prior_names = [p.name for p in priors]
targets = get_targets(DummyTexDoc())

In [ ]:
analysis_folder = RUN_IDS[PRIMARY_ANALYSIS]
i_max = pd.read_hdf(RUNS_PATH / analysis_folder / "output/results.hdf", "likelihood")["logposterior"].idxmax()
idata = az.from_netcdf(RUNS_PATH / analysis_folder / "output/calib_full_out.nc")
best_params = idata_to_sampleiterator(idata).convert("pandas").loc[i_max].to_dict()
parameters.update(best_params)

### Model inspection

In [ ]:
# epi_model.compartments

In [ ]:
# epi_model.flows

In [ ]:
plot_example_model_matrices(epi_model, parameters).update_layout(height=850)

In [ ]:
epi_model.run(parameters=parameters)

### Model outputs

In [ ]:
plot_single_run_outputs(epi_model, targets)

In [ ]:
plot_infection_processes(epi_model.get_derived_outputs_df(), targets, 'notifications_ma')